# Análise descritiva

Este componente realiza a análise descritiva de cada coluna do dataset fornecido. Para colunas numéricas, são calculadas as seguintes estatísticas: `count`, `mean`, `std`, `min`, `max`, além dos percentis. Para dados categóricos, por outro lado, calculam-se `count`, `unique`, `top` e `freq`. O valor de `top` representa a **moda** , enquanto `freq` é a frequência da **moda**.

## Parâmetro de entrada

In [ ]:
# parameters
dataset = "boston" #@param {type:"string"}

## Carrega o dataset juntamente com os respectivos metadados

Carrega o dataset em um `pandas.DataFrame`. Os tipos de cada coluna do dataset são carregados em `featuretypes`.

In [ ]:
from platiagro import load_dataset
from platiagro import stat_dataset

df = load_dataset(name=dataset)
featuretypes = stat_dataset(name=dataset)["featuretypes"]

## Realiza a análise descritiva

In [ ]:
import numpy as np
from platiagro import CATEGORICAL
from platiagro import NUMERICAL

for i, col in enumerate(df):
    if featuretypes[i] == CATEGORICAL:
        df[col] = df[col].astype('category')

def get_description(df, col_type):
    idx = np.where(np.array(featuretypes) == col_type)[0]
    if len(idx) > 0:
        return df.iloc[:, idx].describe().T
    return None

categorical_description = get_description(df, CATEGORICAL)
numeric_description = get_description(df, NUMERICAL)

## Salva as tabelas com as estatísticas da análise descritiva

In [ ]:
import matplotlib.pyplot as plt
from platiagro import save_figure

def render_table(data, col_width=3.0, row_height=0.625, font_size=8,
                 header_color='#40466e', row_colors=['#f1f1f2', 'w'],
                 edge_color='w', bbox=[0, 0, 1, 1], header_columns=0,
                 ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) +\
                np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    # Format table numbers as string
    cellText = [[j[0:20] if type(j) == str else '%.2f' % j for j in i] for i in data.values]
    
    mpl_table = ax.table(
        cellText=cellText, bbox=bbox,
        colLabels=data.columns, 
        rowLabels=data.index,
        **kwargs
    )

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0] % len(row_colors) ])
    return ax

if categorical_description is not None:
    render_table(categorical_description.T.head(20), header_columns=0, col_width=2.0)
    save_figure(figure=plt.gcf())
    plt.clf()

if numeric_description is not None:
    render_table(numeric_description.T.head(20), header_columns=0, col_width=2.0)
    save_figure(figure=plt.gcf())
    plt.clf()